# Transcription of Local File:
Using the microphone and a .wav file of over 278400 bytes (0.278 MB),
Rev.ai's speech to text sdk can be used to transcribe text. 
In this example, the source audio file for the Pynq labs was used.
## Importing Relavent Packages:

In [1]:
from rev_ai import apiclient
from time import sleep

import os
from os import path
import wave
import json
import numpy as np

## Defining Relavent Tokens and Filepaths:

In [4]:
token = "020xN8wEvpFJ57K5xz4CjnhKVkC0kDJO74fKvx58chPRJQHUChMfrQyTWWooMnfO5H5kyGiVdHJHSroppSQFIU9g69v2E"
filepath_ext = "./../../base/audio/recording_0.wav" # Submit a local file for transcription
filepath = "./TestData/TheRaven.wav" # file contained within the same cwd

## Loading Audio File into Jupyter Notebook:

In [3]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
pAudio = base.audio
pAudio.select_microphone()

In [5]:
%%time

with wave.open(filepath, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()

CPU times: user 0 ns, sys: 75.1 ms, total: 75.1 ms
Wall time: 435 ms


## Contacting Rev.ai's SDK to Transcribe Video:

In [14]:
def localfile(token, filepath):
    # initialize Rev AI API client
    client = apiclient.RevAiAPIClient(token)

    # submit a file for transcription
    job = client.submit_job_local_file(filepath)

    # get job id
    job_id = job.id
    print("Job submitted with id: " + job_id)

    # check job status
    while (job.status.name == 'IN_PROGRESS'):
        details = client.get_job_details(job_id)
        print("Job status: " + details.status.name)

        # if successful, print result
        if (details.status.name == 'TRANSCRIBED'):
            # print(client.get_transcript_json(job_id))
            # print(client.get_transcript_text(job_id))
            json_transcript = client.get_transcript_json(job_id)
            text_transcript = client.get_transcript_text(job_id)
            break

        # if unsuccessful, print error
        if (details.status.name == 'FAILED'):
            print("Job failed: " + details.failure_detail)
            break

        sleep(30)

    return json_transcript, text_transcript

## Writing .json and .txt files to an output:

In [15]:
def file_generation(filepath, json_transcript, text_transcript):

    # Split the path in head and tail pair 
    head_tail = os.path.split(filepath)

    # print tail part of the path
    head = str(head_tail[1])
    Head = head.split(".")

    # separate into json and text files
    audio_file_json = Head[0]+'.json'
    audio_file_txt = Head[0]+'.txt'

    cwd = os.getcwd()
    json_audiofilename = os.path.join(cwd, str(audio_file_json))
    txt_audiofilename = os.path.join(cwd, str(audio_file_txt))

    # Writing contents to outfiles:
    with open(json_audiofilename, "w") as outfile:
        json.dump(json_transcript, outfile)

    with open(txt_audiofilename, "w") as outfile:
        outfile.write(text_transcript)
    
    return json_audiofilename, txt_audiofilename

## Calling functions:
Also, printing output for easy comparison for its working.

In [16]:
[json_transcript, text_transcript] = localfile(token, filepath)
[json_audiofilename, txt_audiofilename] = file_generation(filepath, json_transcript, text_transcript)
print(text_transcript)

Job submitted with id: 4aYV2CXKmBgLLJPY
Job status: IN_PROGRESS
Job status: TRANSCRIBED
Speaker 0    00:00:02    Months upon the midnight, dreary while I pondered weak in weary over many acquainting curious volume of forgotten law while I nodded nearly napping. Suddenly there came a tapping as of someone gently wrapping. Wrapping it, my chamber door, dear some visitor Imed tapping at my chamber door. Only this and nothing more. Ah, distinctly, I remember it was in the bleak December, and it's separate dying Ember brought its ghost upon the floor. Eagerly. I wish the Mor ve had, I saw two burrow from my book's, Cecila Sorrow. Sorrow for the Lost, Leno for the Ram. Radiant Maton, whom the angels named Lenore Nameless here forevermore and. 

